In [1]:
import pandas as pd
ruta_dataset = "./movies_dataset.csv"
df = pd.read_csv(ruta_dataset)

C:\Users\User\AppData\Local\Temp\ipykernel_5276\2908874164.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ruta_dataset)


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [3]:
#eliminacion columnas solicitadas
columnas_a_eliminar = ['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage']
df = df.drop(columns=columnas_a_eliminar)

In [4]:
#rellenar con 0 en las columnas solicitadas
df[['revenue', 'budget']] = df[['revenue', 'budget']].fillna(0)
print(df[['revenue', 'budget']].isnull().sum())

revenue    0
budget     0
dtype: int64


In [5]:
#eliminacion de valores nulos de la columna solicitada
df = df.dropna(subset=['release_date'])

In [6]:
# Convertir 'revenue' y 'budget' a numérico, reemplazando valores no válidos con NaN
df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce')
df['budget'] = pd.to_numeric(df['budget'], errors='coerce')

# Crear la columna 'return' con revenue / budget, asignando 0 donde no se pueda calcular
df['return'] = df.apply(lambda row: row['revenue'] / row['budget'] 
                        if pd.notna(row['budget']) and row['budget'] > 0 
                        else 0, axis=1)

In [7]:
# Asegurarse de que la columna 'release_date' esté en formato de fecha (AAAA-mm-dd)
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

In [8]:
# Eliminar valores nulos en 'release_date' (fechas no válidas)
df = df.dropna(subset=['release_date'])

In [9]:
# Crear la columna 'release_year' extrayendo el año de la fecha de estreno
df['release_year'] = df['release_date'].dt.year

# Verificar los resultados
print(df[['release_date', 'release_year']].head())

  release_date  release_year
0   1995-10-30          1995
1   1995-12-15          1995
2   1995-12-22          1995
3   1995-12-22          1995
4   1995-02-10          1995


In [10]:
#desanidar columna belongs_to_collection
import ast
def extract_collection_info(row):
    
    if pd.isna(row):
        return None, None
    try:
        collection = ast.literal_eval(row) if isinstance(row, str) else row
        if isinstance(collection, dict):
            return collection.get('id'), collection.get('name')
    except (ValueError, SyntaxError):
        return None, None

In [11]:
# Aplicar la función
df[['collection_id', 'collection_name']] = df['belongs_to_collection'].apply(
    extract_collection_info).apply(pd.Series)
#desanidar columna production_companies
def extract_company_names(row):
    if pd.isna(row):
        return None
    try:
        companies = ast.literal_eval(row) if isinstance(row, str) else row
        if isinstance(companies, list):
            return [company.get('name') for company in companies if isinstance(company, dict) and 'name' in company]
    except (ValueError, SyntaxError):
        return None

In [12]:
# Aplicar la función
df['production_company_names'] = df['production_companies'].apply(extract_company_names)
#desanidar columna genres
def extract_genre_names(row):
    if pd.isna(row):
        return None
    try:
        genres = ast.literal_eval(row) if isinstance(row, str) else row
        if isinstance(genres, list):
            return [genre.get('name') for genre in genres if isinstance(genre, dict) and 'name' in genre]
    except (ValueError, SyntaxError):
        return None

In [13]:
# Aplicar la función a la columna 'genres'
df['genre_names'] = df['genres'].apply(extract_genre_names)

In [14]:
# Guardar el DataFrame en un archivo CSV
df.to_csv("movies_dataset_final.csv", index=False)

print("Archivo CSV guardado exitosamente como 'movies_dataset_final.csv'.")

Archivo CSV guardado exitosamente como 'movies_dataset_final.csv'.
